# Import modules

In [ ]:
import stlab
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
import time
import sys 
import io
import os
from IPython.display import display, Javascript
from shutil import *
from stlab.devices.RS_SGS100A import RS_SGS100A
from stlab.devices.Keysight_N5183B import Keysight_N5183B

from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration_BMDevice import config, RR_1_IF,RO_lo, readout_len, R1_RS

# Edit configuration parameters

**To Do**

# Define microwave sources

In [ ]:
RR = RS_SGS100A("TCPIP::169.254.184.193::INSTR", reset=True,verb=True) 
RR.EXTref()
RR.RFon()
RR.setCWpower(-15)
RR.setCWfrequency(RO_lo)
RR.write(':SOURce:IQ:IMPairment:LEAKage:I ' + R1_RS[0])
RR.write('SOURce:IQ:IMPairment:LEAKage:Q ' + R1_RS[1])
RR.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + R1_RS[2])
RR.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + R1_RS[3])
RR.IQon()
RR.write(':SOURce:IQ:IMPairment:STATe ON')

#LO for downconversion
MXG = Keysight_N5183B(addr='TCPIP::192.168.1.91::INSTR',reset=True,verb=True)
MXG.RFon()
MXG.setCWpower(19)
MXG.setCWfrequency(RO_lo)
MXG.INTref()

# Configure QM unit

In [ ]:
qmm = QuantumMachinesManager()
qm = qmm.open_qm(config)

# Single Readout Pulse

Send I and Q pulses through the fridge and back to the ADC with parameters defined for the specific element in the loop you enter. You can either obtain the data stream as the raw ADC value in time, then plot and determine the TOF and offset, or you can .fft() the data and see the frequency components.

In [ ]:
a = 0.5 #0.24
n_max = 10000

with program() as tof_cal:
    
    n = declare(int)
    adc_stream = declare_stream(adc_trace=True)                
    frame_rotation(np.pi/4, "RR_1")                                #Why do we need this line?
    
    with for_(n, 0, n<n_max, n+1):
        #update_frequency("RR_1", f_vec[0])    
        reset_phase("RR_1")
        measure('readout'*amp(a), 'RR_1', adc_stream)
        wait(int(1500/4),'RR_1')                       #wait(int((readout_len-readout_len/2-250)/4), "RR_1") #wait(int(16/4),'RR_1')
        
#     #Save raw ADC stream
    with stream_processing():
        adc_stream.input1().average().save("adc1")
        adc_stream.input2().average().save("adc2")    
    
    #Save fft of ADC stream
#     with stream_processing():
#         adc_stream.input1().average().fft().save("adc1")
#         adc_stream.input2().average().fft().save("adc2")

In [ ]:
job = qm.execute(tof_cal, duration_limit=0, data_limit=0) 
res_handles= job.result_handles
#this line below waits for the entire job to stop processing on the OPX before continuing
res_handles.wait_for_all_values()

### Analysis

In [ ]:
adc1_handle = res_handles.get('adc1')
adc1_data=adc1_handle.fetch_all() #fetches an array of data from the handle that the streamed data was sent to

adc2_handle = res_handles.get('adc2')
adc2_data=adc2_handle.fetch_all()

#### Raw ADC 

In [ ]:
plt.figure(num=None, figsize=(10, 8), dpi=100)
plt.axhline(0, lw = 1, color ='black', ls = '--')
plt.plot(adc1_data/2**12,label = 'adc1') #for plotting the raw ADC
plt.plot(adc2_data/2**12,label = 'adc2')
plt.xlabel('Time (ns)', fontsize = 16)
plt.ylabel('ADC', fontsize = 16)
plt.title('One Amp, A={}'.format(a))
plt.xticks(fontsize = 16);plt.yticks(fontsize = 16);
plt.legend(fontsize = 14);

In [ ]:
adc1_offset = np.mean(adc1_data/2**12)
adc2_offset = np.mean(adc2_data/2**12)

print(adc1_offset)
print(adc2_offset)

#### FFT

In [ ]:
#fft
adc1_handle = res_handles.get('adc1')
adc1_data=adc1_handle.fetch_all() #fetches an array of data from the handle that the streamed data was sent to

adc2_handle = res_handles.get('adc2')
adc2_data=adc2_handle.fetch_all()


f_vec = np.fft.fftfreq(readout_len-3, 1e-9)*10**(-6)

adc1_data_re = (adc1_handle.fetch_all()[2:-1].T[0])
adc1_data_im = (adc1_handle.fetch_all()[2:-1].T[1])

print(np.shape(adc1_data_re))
sig = adc1_data_re +1j*adc1_data_im
ph=np.arctan(adc1_data_im/adc1_data_re)

adc2_data_re = (adc2_handle.fetch_all()[2:-1].T[0])
adc2_data_im = (adc2_handle.fetch_all()[2:-1].T[1])

sig2=adc2_data_re +1j*adc2_data_im
ph2=np.arctan(adc2_data_im/adc2_data_re)

fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,4),dpi=100)
ax1.plot(f_vec,adc1_data_re, label='re')
ax1.plot(f_vec,adc1_data_im, label='im')
ax1.set_title('FFT ADC1')
ax1.set_xlabel('Frequency (MHz)')
ax1.legend(loc='best');

ax2.plot(f_vec, adc2_data_re, label='re')
ax2.plot(f_vec, adc2_data_im, label='im')
ax2.set_title('FFT ADC2')
ax2.set_xlabel('Frequency (MHz)')
ax2.legend(loc='best');

In [ ]:
print(len(adc1_data_re))

In [ ]:
prefix = 'S' #prefix for measurement folder name.  Can be anything or empty
idstring = f'R1_Ringdown_fft'

data=np.asarray([adc1_data_re.T,adc1_data_im.T, adc2_data_re.T,adc2_data_im.T])

data_dict={'adc1_re':data[0],
           'adc1_im':data[1],
           'adc2_re':data[2],
           'adc2_im':data[3]
    }

old_stdout = sys.stdout
new_stdout = io.StringIO()
sys.stdout = new_stdout

myfile=stlab.newfile(prefix,idstring,data_dict.keys(),autoindex=True)
output = new_stdout.getvalue()
sys.stdout = old_stdout
print(output)
M_ind = output.find("Measurement Name")
M_name = output[M_ind+len('Measurement Name:  '):-1]

stlab.savedict(myfile,data_dict)

In [ ]:
#plt.figure()
#plt.plot(adc1_handle.fetch_all(),  adc2_handle.fetch_all())
#plt.axis('equal')

## Save this file and configuration file to measurement folder

In [ ]:
#save notebook
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
#define document paths
meas_path = os.path.join(os.getcwd(),M_name)

current_nb_path = os.path.join(os.getcwd(),nb_name)
save_nb_path = os.path.join(meas_path,nb_name)

current_config_path = os.path.join(os.getcwd(), 'Configuration_KNO1.py')
save_config_path = os.path.join(meas_path, 'Configuration_KNO1.py')

#copy to measurement folder 
copy2(current_nb_path,save_nb_path);
copy2(current_config_path,save_config_path);